# # setup
---

In [ ]:
%cd /data/kimgh/CenterPoint-custom/CenterPoint-static/det3d/ops/iou3d_nms

!python setup.py build_ext --inplace

# # Prepare data
---

In [ ]:
%cd /data/kimgh/CenterPoint

!python create_data.py nuscenes_data_prep --root_path '/data/kimgh/CenterPoint/data/nuScenes/' --version 'v1.0-mini' --nsweeps 10

In [ ]:
%cd /data/kimgh/CenterPoint-custom/CenterPoint-dynamic

root_path = '/data/kimgh/CenterPoint-custom/CenterPoint-dynamic/data'
sensor ='lidar'

!python create_data.py nia_data_prep --root_path {root_path} --sensor {sensor}

In [ ]:
%cd /data/kimgh/CenterPoint-custom/CenterPoint-static

root_path = '/data/kimgh/CenterPoint-custom/CenterPoint-static/data/lidar'
sensor = 'lidar'
subsample = 10
num_process = 10

f'!python create_data.py nia_data_prep --root_path {root_path} --sensor {sensor} --subsample {subsample} --num_process {num_process}'

In [ ]:
%cd /data/kimgh/CenterPoint-custom/CenterPoint-static

root_path = '/data/kimgh/CenterPoint-custom/CenterPoint-static/data/selectsub4'
sensor = 'radar'
subsample = 10
num_process = 20

f'!python create_data.py nia_data_prep --root_path {root_path} --sensor {sensor} --num_process {num_process}'

In [ ]:
%cd /data/kimgh/CenterPoint-custom/CenterPoint-static

root_path = '/data/kimgh/CenterPoint-custom/CenterPoint-static/data/etc/test'
sensor = 'lidar'
subsample = 10
num_process = 10

!python create_data.py nia_data_prep --root_path {root_path} --sensor {sensor} --subsample {subsample} --num_process {num_process}

In [ ]:
%cd /data/kimgh/CenterPoint-custom/CenterPoint-static

root_path = '/data/kimgh/CenterPoint-custom/CenterPoint-static/data/sample'
sensor = 'radar'
subsample = False
num_process = 10

!python create_data.py nia_data_prep --root_path {root_path} --sensor {sensor} --subsample {subsample} --num_process {num_process}

# # Train
---
- pm2 start train_multigpu.sh --interpreter bash --name "centerpoint_train_lidar" --no-autorestart -- 4 --config /data/kimgh/CenterPoint-custom/CenterPoint-static/configs/nia/lidar/nia_centerpoint_voxelnet_01voxel_lidar_train.py --work_dir ./result/lidar_1stsub/train --validate

- CUDA_VISIBLE_DEVICES=1,2 pm2 start train_multigpu.sh --interpreter bash --name "centerpoint_train_radar" --no-autorestart -- 2 --config /data/kimgh/CenterPoint-custom/CenterPoint-static/configs/nia/radar/nia_centerpoint_voxelnet_01voxel_radar_train.py --work_dir ./result/selectsub/train/radar --validate

In [ ]:
%cd /data/kimgh/CenterPoint

CONFIG_PATH = '/data/kimgh/CenterPoint/configs/nusc/voxelnet/nusc_centerpoint_voxelnet_01voxel.py'
work_dir = './result/nusc_centerpoint_voxelnet_01voxel'

f'!python -m torch.distributed.launch --nproc_per_node 4 train.py {CONFIG_PATH} --work_dir {work_dir} --validate'

In [1]:
%cd /data/kimgh/CenterPoint-custom/CenterPoint-static

CONFIG_PATH = '/data/kimgh/CenterPoint-custom/CenterPoint-static/configs/nia/lidar/nia_centerpoint_voxelnet_01voxel_lidar_train.py'
work_dir = './result/selectsub3/train/lidar'
resume_from = '/data/kimgh/CenterPoint-custom/CenterPoint-static/result/selectsub3/train/lidar/latest.pth'

# f'!python -m torch.distributed.launch --nproc_per_node 4 train.py --config {CONFIG_PATH} --work_dir {work_dir} --validate'
f'!bash train_multigpu.sh 3 --config {CONFIG_PATH} --work_dir {work_dir} --validate --resume_from {resume_from}'

/data/kimgh/CenterPoint-custom/CenterPoint-static


'!bash train_multigpu.sh 3 --config /data/kimgh/CenterPoint-custom/CenterPoint-static/configs/nia/lidar/nia_centerpoint_voxelnet_01voxel_lidar_train.py --work_dir ./result/selectsub3/train/lidar --validate --resume_from /data/kimgh/CenterPoint-custom/CenterPoint-static/result/selectsub3/train/lidar/latest.pth'

In [4]:
%cd /data/kimgh/CenterPoint-custom/CenterPoint-static

CONFIG_PATH = '/data/kimgh/CenterPoint-custom/CenterPoint-static/configs/nia/radar/nia_centerpoint_voxelnet_01voxel_radar_train.py'
work_dir = './result/selectsub4/train/radar'
resume_from = '/data/kimgh/CenterPoint-custom/CenterPoint-static/result/selectsub4/train/radar/latest.pth'

# f'!python -m torch.distributed.launch --nproc_per_node 4 train.py --config {CONFIG_PATH} --work_dir {work_dir} --validate'
f'!bash train_multigpu.sh 3 --config {CONFIG_PATH} --work_dir {work_dir} --validate --resume_from {resume_from}'

/data/kimgh/CenterPoint-custom/CenterPoint-static


'!bash train_multigpu.sh 3 --config /data/kimgh/CenterPoint-custom/CenterPoint-static/configs/nia/radar/nia_centerpoint_voxelnet_01voxel_radar_train.py --work_dir ./result/selectsub4/train/radar --validate --resume_from /data/kimgh/CenterPoint-custom/CenterPoint-static/result/selectsub4/train/radar/latest.pth'

In [ ]:
%cd /data/kimgh/CenterPoint-custom/CenterPoint-dynamic

CONFIG_PATH = '/data/kimgh/CenterPoint-custom/CenterPoint-dynamic/configs/nia/lidar/nia_centerpoint_voxelnet_01voxel.py'
work_dir = './result/selectsub/train/lidar'

!python -m torch.distributed.launch --nproc_per_node 4 train.py --config {CONFIG_PATH} --work_dir {work_dir} --validate

# # Validation
---

In [2]:
# lidar-normal

%cd /data/kimgh/CenterPoint-custom/CenterPoint-static

data = 'selectsub3'
epoch_num = 70
sensor = 'lidar'

config = f'/data/kimgh/CenterPoint-custom/CenterPoint-static/configs/nia/{sensor}/nia_centerpoint_voxelnet_01voxel_{sensor}_normal.py'
work_dir = f'/data/kimgh/CenterPoint-custom/CenterPoint-static/result/{data}/test/{sensor}/normal_epoch{epoch_num}'
checkpoint = f'/data/kimgh/CenterPoint-custom/CenterPoint-static/result/{data}/train/{sensor}/epoch_{epoch_num}.pth'
# sensor = 'lidar'

# f'!CUDA_VISIBLE_DEVICES=0 python dist_test.py --config {config} --work_dir {work_dir} --checkpoint {checkpoint} --sensor {sensor} --speed_test'
f'!CUDA_VISIBLE_DEVICES=0,1,2,3 bash test_multigpu.sh 4 --config {config} --work_dir {work_dir} --checkpoint {checkpoint} --sensor {sensor} --speed_test'

/data/kimgh/CenterPoint-custom/CenterPoint-static


'!CUDA_VISIBLE_DEVICES=0 python dist_test.py --config /data/kimgh/CenterPoint-custom/CenterPoint-static/configs/nia/lidar/nia_centerpoint_voxelnet_01voxel_lidar_normal.py --work_dir /data/kimgh/CenterPoint-custom/CenterPoint-static/result/selectsub3/test/lidar/normal_epoch70 --checkpoint /data/kimgh/CenterPoint-custom/CenterPoint-static/result/selectsub3/train/lidar/epoch_70.pth --sensor lidar --speed_test'

In [ ]:
# lidar-abnormal

%cd /data/kimgh/CenterPoint-custom/CenterPoint-static

data = 'selectsub3'
epoch_num = 36
sensor = 'lidar'

config = f'/data/kimgh/CenterPoint-custom/CenterPoint-static/configs/nia/{sensor}/nia_centerpoint_voxelnet_01voxel_{sensor}_abnormal.py'
work_dir = f'/data/kimgh/CenterPoint-custom/CenterPoint-static/result/{data}/test/{sensor}/abnormal_epoch{epoch_num}'
checkpoint = f'/data/kimgh/CenterPoint-custom/CenterPoint-static/result/{data}/train/{sensor}/epoch_{epoch_num}.pth'
# sensor = 'lidar'

# f'!CUDA_VISIBLE_DEVICES=1 python dist_test.py --config {config} --work_dir {work_dir} --checkpoint {checkpoint} --sensor {sensor} --speed_test'
f'!CUDA_VISIBLE_DEVICES=0,1,2,3 bash test_multigpu.sh 4 --config {config} --work_dir {work_dir} --checkpoint {checkpoint} --sensor {sensor} --speed_test'

In [1]:
# radar-normal

%cd /data/kimgh/CenterPoint-custom/CenterPoint-static

data = 'selectsub4'
epoch_num = 18
sensor = 'radar'

config = f'/data/kimgh/CenterPoint-custom/CenterPoint-static/configs/nia/{sensor}/nia_centerpoint_voxelnet_01voxel_{sensor}_normal.py'
work_dir = f'/data/kimgh/CenterPoint-custom/CenterPoint-static/result/{data}/test/{sensor}/normal_epoch{epoch_num}'
checkpoint = f'/data/kimgh/CenterPoint-custom/CenterPoint-static/result/{data}/train/{sensor}/epoch_{epoch_num}.pth'

# f'!CUDA_VISIBLE_DEVICES=2 python dist_test.py --config {config} --work_dir {work_dir} --checkpoint {checkpoint} --sensor {sensor} --speed_test'
f'!CUDA_VISIBLE_DEVICES=0,1,2 bash test_multigpu.sh 3 --config {config} --work_dir {work_dir} --checkpoint {checkpoint} --sensor {sensor} --speed_test'

/data/kimgh/CenterPoint-custom/CenterPoint-static


'!CUDA_VISIBLE_DEVICES=0,1,2 bash test_multigpu.sh 3 --config /data/kimgh/CenterPoint-custom/CenterPoint-static/configs/nia/radar/nia_centerpoint_voxelnet_01voxel_radar_normal.py --work_dir /data/kimgh/CenterPoint-custom/CenterPoint-static/result/selectsub4/test/radar/normal_epoch18 --checkpoint /data/kimgh/CenterPoint-custom/CenterPoint-static/result/selectsub4/train/radar/epoch_18.pth --sensor radar --speed_test'

In [2]:
# radar-abnormal

%cd /data/kimgh/CenterPoint-custom/CenterPoint-static

data = 'selectsub4'
epoch_num = 18
sensor = 'radar'

config = f'/data/kimgh/CenterPoint-custom/CenterPoint-static/configs/nia/{sensor}/nia_centerpoint_voxelnet_01voxel_{sensor}_abnormal.py'
work_dir = f'/data/kimgh/CenterPoint-custom/CenterPoint-static/result/{data}/test/{sensor}/abnormal_epoch{epoch_num}'
checkpoint = f'/data/kimgh/CenterPoint-custom/CenterPoint-static/result/{data}/train/{sensor}/epoch_{epoch_num}.pth'

# f'!CUDA_VISIBLE_DEVICES=3 python dist_test.py --config {config} --work_dir {work_dir} --checkpoint {checkpoint} --sensor {sensor} --speed_test'
f'!CUDA_VISIBLE_DEVICES=0,1,2 bash test_multigpu.sh 3 --config {config} --work_dir {work_dir} --checkpoint {checkpoint} --sensor {sensor} --speed_test'

/data/kimgh/CenterPoint-custom/CenterPoint-static


'!CUDA_VISIBLE_DEVICES=0,1,2 bash test_multigpu.sh 3 --config /data/kimgh/CenterPoint-custom/CenterPoint-static/configs/nia/radar/nia_centerpoint_voxelnet_01voxel_radar_abnormal.py --work_dir /data/kimgh/CenterPoint-custom/CenterPoint-static/result/selectsub4/test/radar/abnormal_epoch18 --checkpoint /data/kimgh/CenterPoint-custom/CenterPoint-static/result/selectsub4/train/radar/epoch_18.pth --sensor radar --speed_test'